# Diabetes readmission notebook
Compiled by Victor Ruiz

In [1]:
import pandas as pd

# Ingest the dataset from our github repo
# For more information about the data, see on Kaggle: https://www.kaggle.com/brandao/diabetes#description.pdf

diabetes_data = pd.read_csv('https://raw.githubusercontent.com/arcus/education-materials/master/ml-intermediate/datasets/diabetes/diabetic_data.csv')


In [2]:
diabetes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        101766 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  101766 non-null object
medical_specialty           101766 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            10176

In [3]:
diabetes_data.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


What is the outcome?

In [4]:
diabetes_data.readmitted.value_counts()

NO     54864
>30    35545
<30    11357
Name: readmitted, dtype: int64

In [5]:
### convert class to binary
diabetes_data.loc[:, 'readmitted'] = diabetes_data.readmitted == '<30' # but why?
diabetes_data.readmitted.value_counts()

False    90409
True     11357
Name: readmitted, dtype: int64

Things to note from the data above:


*   There ARE missing data
*   Object columns have binary and categorical values

Let's replace '?' with np.nan and identify boolean features

In [6]:
import numpy as np
diabetes_data = diabetes_data.replace('?', np.nan)
binary_features = [
      var for var in diabetes_data.columns if diabetes_data[var].isin(['Yes', 'No', np.nan]).all()
]

In [7]:
binary_features

['examide', 'citoglipton', 'diabetesMed']

In [8]:
diabetes_data.tolazamide.value_counts() # see anything problematic????

No        101727
Steady        38
Up             1
Name: tolazamide, dtype: int64

In [9]:
diabetes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        99493 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      3197 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  61510 non-null object
medical_specialty           51817 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            101766 non

So we hace both numeric and categorical features, what do we do about that? --> Encoding! (dummy, one-hot, ordinal, ..., )

sklearn requires numeric features only, so we will use dummy encoding for non-numeric features

In [10]:
numeric = diabetes_data.dtypes[(diabetes_data.dtypes == 'int') | (diabetes_data.dtypes == 'float')].index.values
non_numeric = diabetes_data.columns[~diabetes_data.columns.isin(numeric)].values

non_numeric

array(['race', 'gender', 'age', 'weight', 'payer_code',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone',
       'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide',
       'citoglipton', 'insulin', 'glyburide-metformin',
       'glipizide-metformin', 'glimepiride-pioglitazone',
       'metformin-rosiglitazone', 'metformin-pioglitazone', 'change',
       'diabetesMed', 'readmitted'], dtype=object)

Just in case, let's check how many unique values each of these feature has

In [11]:
diabetes_data[non_numeric].apply(lambda x: x.unique().size, axis=0)
diabetes_data.diag_1.value_counts().head(20)

428      6862
414      6581
786      4016
410      3614
486      3508
427      2766
491      2275
715      2151
682      2042
434      2028
780      2019
996      1967
276      1889
38       1688
250.8    1680
599      1595
584      1520
V57      1207
250.6    1183
518      1115
Name: diag_1, dtype: int64

For simplicity, let's assume that diag_1 is the primary diagnosis and drop the rest

In [12]:
del diabetes_data['diag_2']
del diabetes_data['diag_3'] # yes, I know how ugly this looks

Are numeric features really numeric?


In [13]:
diabetes_data[numeric].head()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,2278392,8222157,6,25,1,1,41,0,1,0,0,0,1
1,149190,55629189,1,1,7,3,59,0,18,0,0,0,9
2,64410,86047875,1,1,7,2,11,5,13,2,0,1,6
3,500364,82442376,1,1,7,2,44,1,16,0,0,0,7
4,16680,42519267,1,1,7,1,51,0,8,0,0,0,5




*   encounter_id should not be used for prediction, why?
*   ^ditto for patient_nbr
* What about admission_type_id, discharge_disposition_id, admission_source_id?



In [14]:
non_numeric = np.concatenate((non_numeric, np.array(['admission_type_id', 'discharge_disposition_id', 'admission_source_id'])))
del diabetes_data['encounter_id'] # still ugly -.-'
del diabetes_data['patient_nbr']

OK, encoding time!

In [15]:
non_numeric = non_numeric[~np.isin(non_numeric, ['diag_2', 'diag_3', 'readmitted', 'encounter_id', 'patient_nbr'])]
encoded_data = pd.get_dummies(data=diabetes_data, prefix_sep='__dummycat__', dummy_na=True, columns=non_numeric, drop_first=True)

In [16]:
encoded_data.shape

(101766, 975)

In [17]:
diabetes_data.shape

(101766, 46)

In [18]:
encoded_data.head()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted,race__dummycat__Asian,...,admission_source_id__dummycat__9.0,admission_source_id__dummycat__10.0,admission_source_id__dummycat__11.0,admission_source_id__dummycat__13.0,admission_source_id__dummycat__14.0,admission_source_id__dummycat__17.0,admission_source_id__dummycat__20.0,admission_source_id__dummycat__22.0,admission_source_id__dummycat__25.0,admission_source_id__dummycat__nan
0,1,41,0,1,0,0,0,1,False,0,...,0,0,0,0,0,0,0,0,0,0
1,3,59,0,18,0,0,0,9,False,0,...,0,0,0,0,0,0,0,0,0,0
2,2,11,5,13,2,0,1,6,False,0,...,0,0,0,0,0,0,0,0,0,0
3,2,44,1,16,0,0,0,7,False,0,...,0,0,0,0,0,0,0,0,0,0
4,1,51,0,8,0,0,0,5,False,0,...,0,0,0,0,0,0,0,0,0,0


Clean garbage from encoding

In [ ]:
nunique = encoded_data.nunique()
constant_features = nunique[nunique == 1].index.values
constant_features

In [ ]:
encoded_data = encoded_data.reindex(encoded_data.columns[~encoded_data.columns.isin(constant_features)], axis=1)

OK time for machine learning -.-'

# Set up cross-validation

In [ ]:
### create 5 stratified folds
from sklearn.model_selection import StratifiedKFold
class_labels = encoded_data.readmitted.values
data = encoded_data.values
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
train_sets = []
test_sets = []

#split data between variables and outcome
X, y = encoded_data[encoded_data.columns[encoded_data.columns != 'readmitted']].copy(), encoded_data.readmitted.copy()
for train_index, test_index in skf.split(data, class_labels):
  train_sets += [(X.iloc[train_index].copy(), y.iloc[train_index].copy())]
  test_sets += [(X.iloc[test_index].copy(), y.iloc[test_index].copy())]
  print(train_index.shape, test_index.shape)

What does one fold look like?

In [ ]:
train_sets[0][0]

In [ ]:
train_sets[0][1]

In [ ]:
train_sets[0][0].shape

In [ ]:
test_sets[0][0].shape

# Define preprocessing and classification pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
pipe = Pipeline([
                 ('impute', SimpleImputer(strategy='median')),
                 ('rf', DecisionTreeClassifier())
])

# Fit a decision tree for each fold

In [ ]:
from copy import deepcopy  # to clone pipeline
models = []
class_probs_train = []
class_probs_test = []
from tqdm import tqdm
for ii in tqdm(range(5)):
  X_train, y_train = train_sets[ii]
  X_test, y_test = test_sets[ii]
  model = deepcopy(pipe)
  model.fit(X_train, y=y_train)
  models += [model]
  class_probs_train += [model.predict_proba(X_train)]
  class_probs_test += [model.predict_proba(X_test)]

# Compute AUC, sensitivity, specificity, and PPV

In [ ]:
#how do probabilities look like?
class_probs_train[0]

In [ ]:
class_probs_train[0].shape

In [ ]:
#what about models?
models[0]

In [ ]:
models[0]['rf']

In [ ]:
models[0]['rf'].__dict__

In [ ]:
def get_metrics(y_true, y_score, fold, desc, threshold=0.5):
  from sklearn.metrics import roc_auc_score, confusion_matrix
  import pandas as pd
  tn, fp, fn, tp = confusion_matrix(y_true, y_score >= threshold).ravel()
  sensitivity = tp / (tp + fn)
  specificity = tn / (tn + fp)
  ppv = tp / (tp + fp)
  auc = roc_auc_score(y_true, y_score)
  vals = [sensitivity, specificity, ppv, auc, fold, desc]
  names = ['sensitivity', 'specificity', 'ppv', 'auc', 'fold', 'desc']
  return pd.Series(vals, names)


In [ ]:
metrics = pd.DataFrame()
for ii in tqdm(range(5)):
  probs_train, probs_test = class_probs_train[ii][:, 1], class_probs_test[ii][:, 1]
  y_train, y_test = train_sets[ii][1], test_sets[ii][1]
  metrics_train = get_metrics(y_train, probs_train, ii, 'training')
  metrics_test = get_metrics(y_test, probs_test, ii, 'test')
  metrics = metrics.append(metrics_train, ignore_index=True, sort=False)
  metrics = metrics.append(metrics_test, ignore_index=True, sort=False)

In [ ]:
metrics

# Get average statistics

In [ ]:
metrics.groupby(['desc']).agg([np.mean, np.std])

In [ ]:
# CHALLENGE: Would you like to use a specific or different measure? Play around a bit here if you'd like!

# Results' interpretation
* Perfect classification in training
* Poor classification in test
* Model has high specificity but low sensitivity. How can this be used?
* Very little variation in test performance (still poor though)
* Could model complexity (N features) be causing overfitting?

Let's repeat the experiment with fewer features and see what happens

In [ ]:
# metrics = pd.DataFrame()
# models = []
# from sklearn.feature_selection import SelectKBest, mutual_info_classif
# for ii in tqdm(range(5)):
#   X_train, y_train = train_sets[ii]
#   X_test, y_test = test_sets[ii]
#   model = Pipeline([
#                 ('impute', SimpleImputer(strategy='median')),
#                 ('select', SelectKBest(score_func=mutual_info_classif, k=20)),
#                 ('rf', DecisionTreeClassifier())
#   ])
#   model.fit(X_train, y=y_train)
#   models += [model]
#   probs_train, probs_test = model.predict_proba(X_train)[:, 1], model.predict_proba(X_test)[:, 1]
#   metrics_train = get_metrics(y_train, probs_train, ii, 'training')
#   metrics_test = get_metrics(y_test, probs_test, ii, 'test')
#   metrics = metrics.append(metrics_train, ignore_index=True, sort=False)
#   metrics = metrics.append(metrics_test, ignore_index=True, sort=False)

In [ ]:
keep_features_regex = '^race.+|^gender.+|^age.+|^weight.+|^admission_type_id.+|^time_in_hospital.+|^payer_code.+|^num_lab_procedures.+|^num_procedures.+|^num_medications.+|^number_outpatient.+|^number_emergency.+|^number_inpatient.+|insulin|diabetes_med'

metrics = pd.DataFrame()
models = []
for ii in tqdm(range(5)):
  X_train, y_train = train_sets[ii]
  X_test, y_test = test_sets[ii]
  keep_features = X.columns[X.columns.str.contains(keep_features_regex)].tolist()
  X_train = X_train[keep_features].copy()
  X_test = X_test[keep_features].copy()
  model = Pipeline([
                ('impute', SimpleImputer(strategy='median')),
                ('dt', DecisionTreeClassifier())
  ])
  model.fit(X_train, y=y_train)
  models += [model]
  probs_train, probs_test = model.predict_proba(X_train)[:, 1], model.predict_proba(X_test)[:, 1]
  metrics_train = get_metrics(y_train, probs_train, ii, 'training')
  metrics_test = get_metrics(y_test, probs_test, ii, 'test')
  metrics = metrics.append(metrics_train, ignore_index=True, sort=False)
  metrics = metrics.append(metrics_test, ignore_index=True, sort=False)

In [ ]:
metrics.groupby(['desc']).agg([np.mean, np.std])

## Model still sucks... Let's try with a more complex model



In [ ]:
# keep_features_regex = '^race.+|^gender.+|^age.+|^weight.+|^admission_type_id.+|^time_in_hospital.+|^payer_code.+|^num_lab_procedures.+|^num_procedures.+|^num_medications.+|^number_outpatient.+|^number_emergency.+|^number_inpatient.+|insulin|diabetes_med'
from sklearn.ensemble import RandomForestClassifier
metrics = pd.DataFrame()
models = []
for ii in tqdm(range(5)):
  X_train, y_train = train_sets[ii]
  X_test, y_test = test_sets[ii]
  keep_features = X.columns[X.columns.str.contains(keep_features_regex)].tolist()
  # X_train = X_train[keep_features].copy()
  # X_test = X_test[keep_features].copy()
  model = Pipeline([
                ('impute', SimpleImputer(strategy='median')),
                ('rf', RandomForestClassifier())
  ])
  model.fit(X_train, y=y_train)
  models += [model]
  probs_train, probs_test = model.predict_proba(X_train)[:, 1], model.predict_proba(X_test)[:, 1]
  metrics_train = get_metrics(y_train, probs_train, ii, 'training')
  metrics_test = get_metrics(y_test, probs_test, ii, 'test')
  metrics = metrics.append(metrics_train, ignore_index=True, sort=False)
  metrics = metrics.append(metrics_test, ignore_index=True, sort=False)

In [ ]:
metrics.groupby(['desc']).agg([np.mean, np.std])

# Not stelar but we're getting somewhere. However, using a default prediction threshold of 0.5, we have near-perfect specificity and very poor sensitivity, let's explore how these values change based on the prediction threshold.

In [ ]:
thresholds = np.linspace(0, 1, 100)
metrics = pd.DataFrame([get_metrics(y_test, probs_test, None, var, threshold=var) for var in thresholds])
metrics = metrics.rename(columns={'desc': 'threshold'})


In [ ]:
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(16, 12))
plt.plot( 'threshold', 'sensitivity', data=metrics, marker='o', markerfacecolor='blue', markersize=2, color='skyblue', linewidth=4)
plt.plot( 'threshold', 'specificity', data=metrics, marker='', color='olive', linewidth=2)
plt.plot( 'threshold', 'ppv', data=metrics, marker='', color='black', linewidth=2, linestyle='dashed')
plt.legend()